///---
layout: post
title: "Vim"
comments: true
share: true
date: 2020-05-13 14:00:00
description: Vim 사용을 위한 자료 정리
tags: note
toc: false
sitemap :
    changefreq : daily
    priority : 1.0
///---

# Vim

## 명령어

## 설정

### Neovim 설치

Neovim 최신 안정 버전 설치

```sh
sudo add-apt-repository ppa:neovim-ppa/stable
sudo apt-get update
sudo apt-get install neovim
```

vi를 neovim으로 대체

```sh
sudo update-alternatives --install /usr/bin/vi vi /usr/bin/nvim 60
sudo update-alternatives --config vi 
sudo update-alternatives --install /usr/bin/vim vim /usr/bin/nvim 60 
sudo update-alternatives --config vim 
sudo update-alternatives --install /usr/bin/editor editor /usr/bin/nvim 60
sudo update-alternatives --config editor
```

### Neovim init.vim 설정

Neovim의 설정 파일은 ~/.config/nvim/init.vim에 위치한다.
init.vim에서 vim-plug을 사용할 것이므로 vim-plug을 설치한다.

```bash
sh -c 'curl -fLo "${XDG_DATA_HOME:-$HOME/.local/share}"/nvim/site/autoload/plug.vim --create-dirs \
       https://raw.githubusercontent.com/junegunn/vim-plug/master/plug.vim'
```

그리고 deoplete 사용을 위해 pynvim을 설치한다.

```bash
pip install pynvim
```

https://seulcode.tistory.com/272 을 참고한 [init.vim](/assets/data/vim/init.vim)  
  
UltiSnips의 Python snippets를 ~/.config/nvim/UltiSnips에 복사한다. [python.snippets](/assets/data/vim/python.snippets)

#### COC plugin 설치

Coc 사용을 위해서 nodejs와 npm가 필요하므로 linux에 nodejs와 npm을 설치한 후 vim에서 다음 명령을 실행한다.

```bash
:CocInstall coc-json
:CocInstall coc-python
:CocInstall coc-tabnine
```

### Gvim _vimrc 설정

gvim 설정 파일은 %HOME% 폴더에 위치한다.
_vimrc에서 vim-plugin을 사용할 것이므로 powershell에서 아래 명령을 이용하여 vim-plugin을 설치한다.
gvim에서 64bit python을 사용하기 위해서는 64bit gvim을 찾아서 설치한다. stable 버전에는 64bit gvim이 없으므로 nightly 버전을 github에서 다운받아야 한다.

```powershell
md ~\vimfiles\autoload
$uri = 'https://raw.githubusercontent.com/junegunn/vim-plug/master/plug.vim'
(New-Object Net.WebClient).DownloadFile(
  $uri,
  $ExecutionContext.SessionState.Path.GetUnresolvedProviderPathFromPSPath(
    "~\vimfiles\autoload\plug.vim"
  )
)
```

[_vimrc](/assets/data/vim/_vimrc)  
  
UltiSnips의 Python snippets를 ~/vimfile/UltiSnips에 복사한다. [python.snippets](/assets/data/vim/python.snippets)

### Gvim anaconda 환경에서 설정

anaconda를 active하고 gvim을 실행하기 위해 간단한 실행 파일을 작성하였다. 이 파일을 gvim.exe로 대체하고 gvim.exe는 gvim_.exe로 변경한다.

```c++
// gvim_launcher.cpp : 이 파일에는 'main' 함수가 포함됩니다. 거기서 프로그램 실행이 시작되고 종료됩니다.
//

#include <Windows.h>
#include <stdio.h>
#include <tchar.h>

int main(int argc, TCHAR* argv[])
{
    FreeConsole();
    STARTUPINFO si;
    PROCESS_INFORMATION pi;
    int idx_arg_start;
    int idx_arg;
    const size_t commandline_size = 4096;
    const size_t buff_size = 1024;

    ZeroMemory(&si, sizeof(si));
    si.cb = sizeof(si);
    ZeroMemory(&pi, sizeof(pi));
    TCHAR tcsCommandLine[commandline_size];
    memset(tcsCommandLine, '\0', sizeof(TCHAR) * commandline_size);
    TCHAR tcsAnacondaVim[] = _T("CMD.EXE /C set MPLCURSORS={\"multiple\": 1}&c:\\anaconda3\\scripts\\activate&\"C:\\Program Files (x86)\\Vim\\vim82\\gvim_.exe\"");
    TCHAR tcsDiffArg[] = _T(" --literal -d");
    TCHAR tcsTabArg[] = _T(" --literal -p");
    TCHAR tcsBuff[buff_size];

	swprintf(tcsCommandLine, commandline_size, _T("%s"), tcsAnacondaVim);

    if (argc > 3 && strcmp((char*)argv[2], "-d") == 0)  // For diff
    {
        _tcscat_s(tcsCommandLine, commandline_size, tcsDiffArg);
        idx_arg_start = 3;
    }
    else
    {
        _tcscat_s(tcsCommandLine, commandline_size, tcsTabArg);
        idx_arg_start = 1;
    }

    for (idx_arg = idx_arg_start; idx_arg < argc; idx_arg++)
    {
        _tcscat_s(tcsCommandLine, commandline_size, _T(" "));
        memset(tcsBuff, '\0', sizeof(TCHAR) * buff_size);
        MultiByteToWideChar(CP_ACP, MB_PRECOMPOSED, (char*)argv[idx_arg], strlen((char*)argv[idx_arg]), tcsBuff, buff_size);

        _tcscat_s(tcsCommandLine, commandline_size, tcsBuff);
    }
    _tcscat_s(tcsCommandLine, commandline_size, _T("\0"));

    // Start the child process. 
    if (!CreateProcess(NULL,   // No module name (use command line)
        tcsCommandLine,        // Command line
        NULL,           // Process handle not inheritable
        NULL,           // Thread handle not inheritable
        FALSE,          // Set handle inheritance to FALSE
        CREATE_NO_WINDOW,              // No creation flags
        NULL,           // Use parent's environment block
        NULL,           // Use parent's starting directory 
        &si,            // Pointer to STARTUPINFO structure
        &pi)           // Pointer to PROCESS_INFORMATION structure
        )
    {
        printf("CreateProcess failed (%d).\n", GetLastError());
        return 0;
    }

    // Wait until child process exits.
    WaitForSingleObject(pi.hProcess, INFINITE);

    // Close process and thread handles. 
    CloseHandle(pi.hProcess);
    CloseHandle(pi.hThread);

    return 0;
}

```

[vim.ico](/assets/data/vim/gvim.ico)

### ipython vim keybinding

~/.ipython/profile_default/ipython_config.py에 아래 명령어 추가

```
c.TerminalInteractiveShell.editing_mode = 'vi'
```

### ESC 입력시 자동 영어 변환 (VScode)

Vim에서 한글 입력 후에 ESC를 눌려 Normal 모드로 진입하면 어떤 명령어도 동작되지 않는 문제가 발생한다. 명령어가 영어가 아닌 한글로 입력되면서 벌어지는 문제인데 VScode의 Vim Plugin은 이 문제를 해결하기 위한 방안을 제공한다.

#### Windows

설정 -> 언어 -> 영어(미국) 추가한다.
https://github.com/daipeihust/im-select 에서 [im-select](/assets/data/vim/im-select.zip) 를 다운 받는다.  
Vscode의 환경설정에 다음 내용을 추가한다.

```
"vim.autoSwitchInputMethod.enable": true,
"vim.autoSwitchInputMethod.defaultIM": "1033",
"vim.autoSwitchInputMethod.obtainIMCmd": "D:\\bin\\im-select.exe",
"vim.autoSwitchInputMethod.switchIMCmd": "D:\\bin\\im-select.exe {im}"
```

#### Ubuntu

Nimf 사용자일 경우 Vscode 환경설정에 다음 내용을 추가한다.

```
```

### Capslock -> ESC

#### Windows
AutoHotKey를 설치하고 ahk 확장자 파일을 만들어 아래 내용을 추가하거나 Registry를 변경한다.

- AutoHotKey Script

```
Capslock::Esc
Esc::Capslock
```

- Registry

```
Windows Registry Editor Version 5.00
[HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\Keyboard Layout]
"Scancode Map"=hex:00,00,00,00,00,00,00,00,03,00,00,00,3a,00,01,00,01,00,3a,00,00,00,00,00
```

#### Mint

- 설정 -> 키보드 -> 배치 -> 옵션 -> Capslock 동작 변경
- 위의 방안이 동작 안할 시 .xsession 혹은 .xinirc 혹은 .bashrc에 아래 내용 추가

```
setxkbmap -option caps:swapescape
```

- vscode 환경 설정에 아래 내용 추가

```
"keyboard.dispatch": "keyCode"
```

### IdeaVim 설정

pycharm에서 ideavim을 이용하여 vim 모드를 사용할수 있다. 현재 ideavim의 설정 파일인 .ideavimrc는 아래와 같다.

```
" ----- Options -----
set hls     "hlsearch
set ic      "ignorecase
set is      "incsearch
set scs     "smartcase
set nu      "line number
set rnu     "line relativenumber

" ----- Plugins -----
set commentary
set highlightedyank
set NERDTree
set easymotion

" ----- KeyMap -----
let mapleader=" "
:map <leader>f <Action>(ReformatCode)
:map <leader>d <Action>(Debug)
:map <leader>b <Action>(ToggleLineBreakpoint)
map <leader>e :NERDTree<CR>
```
